In [3]:
import lxml, bs4, requests
from bs4 import BeautifulSoup

In [4]:
resp = requests.get("https://www.hltv.org/matches/2329667/mousesports-vs-natus-vincere-esl-pro-league-season-8-finals")

In [5]:
soup = BeautifulSoup(resp.text, "html.parser")

In [7]:
soup.find_all("td", class_="odds-cell")

[]

In [9]:
1/1.75+1/2.37

0.9933694996986135

In [4]:
def dutch_book_stakes(odds1, odds2, bet = 100):   
    #calculate first stake
    s1 = 1 - odds1/(odds1+odds2)
    s1 = round(s1, 4)*bet
    #calculate second stake
    s2 = bet - s1
    print("    stake | payout")
    print("-" * 20)
    print(f" s1 {s1:.5} | {s1*odds1:3.6}\n s2 {s2:.5} | {s1*odds1:3.6}")
    
    
    
dutch_book_stakes(1.62, 2.57)
dutch_book_stakes(2.37, 1.75)
dutch_book_stakes(3.08, 1.53)

    stake | payout
--------------------
 s1 61.34 | 99.3708
 s2 38.66 | 99.3708
    stake | payout
--------------------
 s1 42.48 | 100.678
 s2 57.52 | 100.678
    stake | payout
--------------------
 s1 33.19 | 102.225
 s2 66.81 | 102.225


In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options

In [2]:
import time

In [92]:
url = "https://www.hltv.org/matches/2329667/mousesports-vs-natus-vincere-esl-pro-league-season-8-finals"


In [12]:

def get_bets(url, driver):
    odds = []
    driver.get(url)
    time.sleep(2)
    for betting_box in driver.find_elements_by_css_selector(".betting"):
        for line in betting_box.find_elements_by_css_selector(".betting_provider"):
            cell_entries = []
            for cell in line.find_elements_by_css_selector("td"):
                cell_entries.append(cell.text)
            if len(cell_entries) > 2 and cell_entries[2] == "-" and "%" not in cell_entries[-1]:
                provider = line.get_attribute("class").split(" ")[-2].split("_")[1]
                odds.append([provider, cell_entries[1], cell_entries[-1]])
        
    print("    provider | Team1 | Team2")
    print("-"*13+"+"+"-"*7+"+"+"-"*6)
    for odd in odds: 
        print(f"{odd[0]:{12}} | {odd[1]:{5}} | {odd[2]:{5}}")
            
    return odds    
        
# driver = webdriver.Chrome("chromedriver_win32/chromedriver.exe")
# driver.get("https://www.hltv.org)")
# odds = get_bets(url, driver)

# driver.quit()

In [7]:
import pandas as pd

In [162]:
df = pd.DataFrame(odds, columns=["provider", "team1", "team2"])
# df = df[df["team1"] != ""]
df["team1"] = pd.to_numeric(df["team1"])
df["team2"] = pd.to_numeric(df["team2"])
t1_max = df["team1"].max()
t2_max = df["team2"].max()
print(f"best odds: team1: {t1_max} team2: {t2_max}")
dutch_book_stakes(t1_max, t2_max)


best odds: team1: 2.41 team2: 2.1
    stake | payout
--------------------
 s1 46.56 | 112.21
 s2 53.44 | 112.21


In [13]:
driver = webdriver.Chrome("chromedriver_win32/chromedriver.exe")
driver.get("https://www.hltv.org")
all_odds = []
matches = []
for match in driver.find_elements_by_css_selector(".hotmatch-box"):
    matches.append(match.get_attribute("href"))

time.sleep(5)
    
for match in matches:
    print(match)
    all_odds.append(get_bets(match, driver))

for odds in all_odds:
    df = pd.DataFrame(odds, columns=["provider", "team1", "team2"])
    # df = df[df["team1"] != ""]
    df["team1"] = pd.to_numeric(df["team1"])
    df["team2"] = pd.to_numeric(df["team2"])
    t1_max = df["team1"].max()
    t2_max = df["team2"].max()
    print(f"best odds: team1: {t1_max} team2: {t2_max}")
    dutch_book_stakes(t1_max, t2_max)
driver.quit()

https://www.hltv.org/matches/2329666/mibr-vs-nrg-esl-pro-league-season-8-finals
    provider | Team1 | Team2
-------------+-------+------
egb          | 1.37  | 2.65 
betway       | 1.07  | 7.00 
lootbet      | 1.07  | 6.90 
ggbet        | 1.08  | 6.88 
bet365       | 1.50  | 2.50 
1xbet        | 1.21  | 4.32 
pinnacle     | 1.29  | 3.35 
buff88       | 1.07  | 7.66 
unibet       | 1.56  | 2.30 
22bet        | 1.35  | 3.15 
betit        | 1.10  | 6.50 
tonybet      | 1.50  | 2.45 
unikrn       | 1.08  | 8.00 
https://www.hltv.org/matches/2329668/renegades-vs-big-esl-pro-league-season-8-finals
    provider | Team1 | Team2
-------------+-------+------
egb          | 2.81  | 1.33 
betway       | 1.90  | 1.80 
lootbet      | 1.85  | 1.85 
ggbet        | 1.99  | 1.75 
bet365       | 2.62  | 1.44 
1xbet        | 3.84  | 1.25 
pinnacle     | 3.97  | 1.22 
buff88       | 2.00  | 1.77 
unibet       | 2.75  | 1.41 
22bet        | 5.20  | 1.15 
betit        | 1.83  | 1.83 
tonybet      | 2.55  | 

    provider | Team1 | Team2
-------------+-------+------
egb          | 1.65  | 2.07 
betway       | 1.80  | 1.90 
ggbet        | 1.78  | 1.95 
bet365       | 1.72  | 2.00 
1xbet        | 1.87  | 1.98 
pinnacle     | 1.82  | 2.00 
buff88       | 1.70  | 2.07 
unibet       | 1.80  | 1.94 
22bet        | 1.87  | 1.98 
betit        | 1.73  | 2.00 
tonybet      | 1.80  | 1.90 
unikrn       | 1.83  | 1.83 
https://www.hltv.org/matches/2329693/kinguin-vs-3dmax-virtuspro-loser-esl-pro-league-season-8-europe-relegation
    provider | Team1 | Team2
-------------+-------+------
https://www.hltv.org/matches/2329695/envy-vs-swole-patrol-cantwinalan-loser-esl-pro-league-season-8-north-america-relegation
    provider | Team1 | Team2
-------------+-------+------
https://www.hltv.org/matches/2329702/masked-vs-furia-esea-mdl-season-29-north-america-relegation
    provider | Team1 | Team2
-------------+-------+------
1xbet        | 2.49  | 1.52 
22bet        | 2.49  | 1.52 
https://www.hltv.org/matches

In [11]:
"%" not in "93.4%"

False